In [2]:
import torch
import numpy as np
import pickle
import json
import tqdm
import os
os.chdir("..")

# Interactiveness AP

Result dir for stage-1: ./exp/bst_interactiveness; Result dir for stage-2: ./exp/bst_hoi.

We report interactiveness AP for stage-1 here.

In [39]:
from binary_evaluation import calc_binary

res = pickle.load(open("./exp/bst_interactiveness/bst_result_binary.pkl", "rb")) # layer3: 37.91
keys, bboxes, score = res["keys"], res["bboxes"], res["scores"]
bin_ap, bin_rec = calc_binary(keys, bboxes, score)
print("interactiveness AP: ", 100 * bin_ap)

interactiveness AP:  38.738927236433526


# Utils

In [ ]:
rare_index = np.array([ 9, 23,28, 45,51, 56,63, 64,67, 71,77, 78,81, 84,85, 91,100,101,105,108,113,128,136,137,150,159,166,167,169,173,180,182,185,189,190,193,196,199,206,207,215,217,223,228,230,239,240,255,256,258,261,262,263,275,280,281,282,287,290,293,304,312,316,318,326,329,334,335,346,351,352,355,359,365,380,382,390,391,392,396,398,399,400,402,403,404,405,406,408,411,417,419,427,428,430,432,437,440,441,450,452,464,470,475,483,486,499,500,505,510,515,518,521,523,527,532,536,540,547,548,549,550,551,552,553,556,557,561,579,581,582,587,593,594,596,597,598,600,]) - 1
rare = np.zeros(600)
rare[rare_index] += 2
def iou(bb1, bb2, debug=False):
    x1 = bb1[2] - bb1[0]
    y1 = bb1[3] - bb1[1]
    if x1 < 0:
        x1 = 0
    if y1 < 0:
        y1 = 0
    
    x2 = bb2[2] - bb2[0]
    y2 = bb2[3] - bb2[1]
    if x2 < 0:
        x2 = 0
    if y2 < 0:
        y2 = 0
    
    xiou = min(bb1[2], bb2[2]) - max(bb1[0], bb2[0])
    yiou = min(bb1[3], bb2[3]) - max(bb1[1], bb2[1])
    if xiou < 0:
        xiou = 0
    if yiou < 0:
        yiou = 0

    if debug:
        print(x1, y1, x2, y2, xiou, yiou)
        print(x1 * y1, x2 * y2, xiou * yiou)
    if xiou * yiou <= 0:
        return 0
    else:
        return xiou * yiou / (x1 * y1 + x2 * y2 - xiou * yiou)

def calc_hit(det, gtbox):
    gtbox = gtbox.astype(np.float64)
    hiou = iou(det[:4], gtbox[:4])
    oiou = iou(det[4:], gtbox[4:])
    return min(hiou, oiou)

def nms(scores, bboxes, keys, thresh=0.6):
    idx = np.argsort(scores)[::-1]
    chosen = np.ones(len(idx))
    for i in range(len(idx)):
        if chosen[idx[i]] > 0:
            hbox = bboxes[idx[i], :4]
            obox = bboxes[idx[i], 4:]
            key  = keys[idx[i]]
            sel  = np.where(keys == key)[0]
            for j in sel:
                if idx[i] != j and iou(hbox, bboxes[j, :4]) > thresh and iou(obox, bboxes[j, 4:]) > thresh:
                    chosen[j] = 0
    index = np.where(chosen > 0)[0]
    dump  = np.where(chosen < 1)[0]
    return index

def calc_ap_ko(scores, bboxes, keys, hoi_id, begin, ko_mask, hit_info=False):
    score = scores[:, hoi_id - begin]
    hit, hit_ko = [], []
    idx = np.argsort(score)[::-1]
    gt_bbox = pickle.load(open('./data/hico_20160224_det/annotations/gt_hoi.pkl', 'rb'))[hoi_id]
    npos = 0
    used = {}
    
    for key in gt_bbox.keys():
        npos += gt_bbox[key].shape[0]
        used[key] = set()
    if len(idx) == 0:
        output = {
            'ap' : 0, 'rec': 0, 'ap_ko': 0, 'rec_ko': 0
        }
        return output
    for i in range(min(len(idx), 19999)):
        pair_id = idx[i]
        bbox = bboxes[pair_id, :]
        key  = keys[pair_id]
        if key in gt_bbox:
            maxi = 0.0
            k    = -1
            for i in range(gt_bbox[key].shape[0]):
                tmp = calc_hit(bbox, gt_bbox[key][i, [0, 2, 1, 3, 4, 6, 5, 7]])
                if maxi < tmp:
                    maxi = tmp
                    k    = i
            if k in used[key] or maxi < 0.5:
                hit.append(0)
                hit_ko.append(0)
            else:
                hit.append(1)
                hit_ko.append(1)
                used[key].add(k)
        else:
            hit.append(0)
            if key in ko_mask:
                hit_ko.append(0)
    bottom = np.array(range(len(hit))) + 1
    hit    = np.cumsum(hit)
    rec    = hit / npos
    prec   = hit / bottom
    ap     = 0.0
    for i in range(11):
        mask = rec >= (i / 10.0)
        if np.sum(mask) > 0:
            ap += np.max(prec[mask]) / 11.0
    if len(hit_ko) == 0:
        output = {
            'ap' : ap, 'rec': np.max(rec), 'ap_ko': 0, 'rec_ko': 0
        }
        return output
    bottom_ko = np.array(range(len(hit_ko))) + 1
    hit_ko    = np.cumsum(hit_ko)
    rec_ko    = hit_ko / npos
    prec_ko   = hit_ko / bottom_ko
    ap_ko     = 0.0
    for i in range(11):
        mask = rec_ko >= (i / 10.)
        if np.sum(mask) > 0:
            ap_ko += np.max(prec_ko[mask]) / 11.
    output = {
        'ap' : ap, 'rec': np.max(rec), 'ap_ko': ap_ko, 'rec_ko': np.max(rec_ko)
    }
    return output

def calc_ap(scores, bboxes, keys, hoi_id, begin):
    score = scores[:, hoi_id - begin]
    hit, hit_ko = [], []
    idx = np.argsort(score)[::-1]
    gt_bbox = pickle.load(open('./data/hico_20160224_det/annotations/gt_hoi.pkl', 'rb'))[hoi_id]
    npos = 0
    used = {}
    
    for key in gt_bbox.keys():
        npos += gt_bbox[key].shape[0]
        used[key] = set()
    if len(idx) == 0:
        output = {
            'ap' : 0, 'rec': 0, 'ap_ko': 0, 'rec_ko': 0
        }
        return output
    for i in range(min(len(idx), 19999)):
        pair_id = idx[i]
        bbox = bboxes[pair_id, :]
        key  = keys[pair_id]
        if key in gt_bbox:
            maxi = 0.0
            k    = -1
            for i in range(gt_bbox[key].shape[0]):
                tmp = calc_hit(bbox, gt_bbox[key][i, [0, 2, 1, 3, 4, 6, 5, 7]])
                if maxi < tmp:
                    maxi = tmp
                    k    = i
            if k in used[key] or maxi < 0.5:
                hit.append(0)
            else:
                hit.append(1)
                used[key].add(k)
        else:
            hit.append(0)
    bottom = np.array(range(len(hit))) + 1
    hit    = np.cumsum(hit)
    rec    = hit / npos
    prec   = hit / bottom
    ap     = 0.0
    for i in range(11):
        mask = rec >= (i / 10.0)
        if np.sum(mask) > 0:
            ap += np.max(prec[mask]) / 11.0

    output = {
        'ap' : ap, 'rec': np.max(rec), 'hit': hit,
    }
    return output

In [6]:
anno  = json.load(open('./data/hico_20160224_det/annotations/test_hico.json'))
verb_mapping = pickle.load(open('./util/verb_mapping.pkl', 'rb'), encoding='latin1')
vocab = pickle.load(open('./util/vocab.pkl', 'rb'))

hico_verb_dict = {
    1: 'adjust', 2: 'assemble', 3: 'block', 4: 'blow', 5: 'board', 6: 'break', 7: 'brush_with', 8: 'buy', 9: 'carry', 10: 'catch', 11: 'chase', 12: 'check', 13: 'clean', 14: 'control', 15: 'cook', 16: 'cut', 17: 'cut_with', 18: 'direct', 19: 'drag', 20: 'dribble', 21: 'drink_with', 22: 'drive', 23: 'dry', 24: 'eat', 25: 'eat_at', 26: 'exit', 27: 'feed', 28: 'fill', 29: 'flip', 30: 'flush', 31: 'fly', 32: 'greet', 33: 'grind', 34: 'groom', 35: 'herd', 36: 'hit', 37: 'hold', 38: 'hop_on', 39: 'hose', 40: 'hug', 41: 'hunt', 42: 'inspect', 43: 'install', 44: 'jump', 45: 'kick', 46: 'kiss', 47: 'lasso', 48: 'launch', 49: 'lick', 50: 'lie_on', 51: 'lift', 52: 'light', 53: 'load', 54: 'lose', 55: 'make', 56: 'milk', 57: 'move', 58: 'no_interaction', 59: 'open', 60: 'operate', 61: 'pack', 62: 'paint', 63: 'park', 64: 'pay', 65: 'peel', 66: 'pet', 67: 'pick', 68: 'pick_up', 69: 'point', 70: 'pour', 71: 'pull', 72: 'push', 73: 'race', 74: 'read', 75: 'release', 76: 'repair', 77: 'ride', 78: 'row', 79: 'run', 80: 'sail', 81: 'scratch', 82: 'serve', 83: 'set', 84: 'shear', 85: 'sign', 86: 'sip', 87: 'sit_at', 88: 'sit_on', 89: 'slide', 90: 'smell', 91: 'spin', 92: 'squeeze', 93: 'stab', 94: 'stand_on', 95: 'stand_under', 96: 'stick', 97: 'stir', 98: 'stop_at', 99: 'straddle', 100: 'swing', 101: 'tag', 102: 'talk_on', 103: 'teach', 104: 'text_on', 105: 'throw', 106: 'tie', 107: 'toast', 108: 'train', 109: 'turn', 110: 'type_on', 111: 'walk', 112: 'wash', 113: 'watch', 114: 'wave', 115: 'wear', 116: 'wield', 117: 'zip',
}

obj_range = [
    (161, 170), (11, 24),   (66, 76),   (147, 160), (1, 10), 
    (55, 65),   (187, 194), (568, 576), (32, 46),   (563, 567), 
    (326, 330), (503, 506), (415, 418), (244, 247), (25, 31), 
    (77, 86),   (112, 129), (130, 146), (175, 186), (97, 107), 
    (314, 325), (236, 239), (596, 600), (343, 348), (209, 214), 
    (577, 584), (353, 356), (539, 546), (507, 516), (337, 342), 
    (464, 474), (475, 483), (489, 502), (369, 376), (225, 232), 
    (233, 235), (454, 463), (517, 528), (534, 538), (47, 54), 
    (589, 595), (296, 305), (331, 336), (377, 383), (484, 488), 
    (253, 257), (215, 224), (199, 208), (439, 445), (398, 407), 
    (258, 264), (274, 283), (357, 363), (419, 429), (306, 313), 
    (265, 273), (87, 92),   (93, 96),   (171, 174), (240, 243), 
    (108, 111), (551, 558), (195, 198), (384, 389), (394, 397), 
    (435, 438), (364, 368), (284, 290), (390, 393), (408, 414), 
    (547, 550), (450, 453), (430, 434), (248, 252), (291, 295), 
    (585, 588), (446, 449), (529, 533), (349, 352), (559, 562)
]

peyre_class_dict = {}
for i in range(81, 81 + 117):
    peyre_class_dict[vocab[i].replace(' ', '_')] = i - 81
perm = list(range(117))
for i in range(1, 118):
    perm[i - 1] = peyre_class_dict[hico_verb_dict[i]]
verb_mapping = verb_mapping[perm, :]

# HOI mAP

To evaluate HOI mAP, the result of stage-2 is first used. 

In [40]:
### prepare data
exp_dir = "bst_hoi"
res   = torch.load('./exp/%s/result.pth'%exp_dir, map_location="cpu")
idx, res = res['preds']

a = {'keys': [], 'scores': [], 'bboxes': [], 'os': [], 'binary':[]}
for key in a.keys():
    for i in range(80):
        a[key].append([])
for i in tqdm.trange(len(res)):
    res[i]['boxes']         = res[i]['boxes'].numpy()
    res[i]['verb_scores']   = res[i]['verb_logits'].numpy()
    res[i]['obj_scores']    = res[i]['obj_scores'].numpy()
    pnum = res[i]['verb_scores'].shape[0]
    obj_scores = res[i]['obj_scores']
    tmp = res[i]['obj_scores'].argsort()[::-1]
    orig_boxes = np.concatenate([res[i]["boxes"][np.arange(pnum)], res[i]["boxes"][np.arange(pnum, pnum * 2)]], axis=1)

    for jj in range(64):
        j = tmp[jj]
        ocls = res[i]['labels'][j + pnum]
        a['keys'][ocls].append(int(anno[idx[i]]['file_name'][-10:-4]))
        a['bboxes'][ocls].append(np.concatenate([res[i]['boxes'][j], res[i]['boxes'][j + pnum]]))
        a['scores'][ocls].append(res[i]['verb_scores'][j, obj_range[ocls][0] - 1:obj_range[ocls][1]])
        a['os'][ocls].append(obj_scores[j])

100%|██████████| 9546/9546 [00:06<00:00, 1391.92it/s]


In [42]:
### apply nms
for key in a.keys():
    for i in range(80):
        a[key][i] = np.array(a[key][i])

nms_index = pickle.load(open("./exp/%s/nms_index.pkl"%exp_dir, "rb"))

ap  = np.zeros(600)
rec = np.zeros(600)
for i in tqdm.trange(80):
    if len(a['scores'][i]) == 0:
        continue
    begin = obj_range[i][0] - 1
    end   = obj_range[i][1]

    score = a['scores'][i] * a['os'][i][:, None]
    bbox  = a['bboxes'][i]
    key   = a['keys'][i]

    score, bbox, key = score[nms_index[i]], bbox[nms_index[i]], key[nms_index[i]]

    for hoi_id in range(begin, end):
        output = calc_ap(score, bbox, key, hoi_id, begin)
        ap[hoi_id], rec[hoi_id] = output["ap"], output["rec"]
print("mAP before nis: ", ap.mean())

100%|██████████| 80/80 [00:52<00:00,  1.54it/s]

mAP before nis:  0.34087173833186757


Then, with NIS applied via the result of stage-1, HOI mAP is further boosted.

For NIS, two predicted sets are matched before negative pairs filtering. It is detailed in Suppl.C in our paper.

In [54]:
### read interactiveness results
exp_dir_binary = "bst_interactiveness"
res_binary   = torch.load('./exp/%s/result.pth'%exp_dir_binary, map_location="cpu")
idx_binary, res_binary = res_binary['preds']
Key2Idx = {}
for i in tqdm.trange(len(res_binary)):
    Key2Idx[int(anno[idx_binary[i]["id"]]['file_name'][-10:-4])] = i

100%|██████████| 9546/9546 [00:00<00:00, 314520.91it/s]


In [ ]:
# set matched
for i in range(80):
    if i % 10 == 0:
        print("processing object %d..."%i)
    if len(a['scores'][i]) == 0:
        continue
    begin = obj_range[i][0] - 1
    end   = obj_range[i][1]

    score = a['scores'][i] * a['os'][i][:, None]
    bbox  = a['bboxes'][i]
    key   = a['keys'][i]

    score, bbox, key = score[nms_index[i]], bbox[nms_index[i]], key[nms_index[i]]

    binary = []
    for j in range(len(key)):
        k, b, s = key[j], bbox[j], score[j]
        binary_result = res_binary[Key2Idx[k]]
        binary_scores = binary_result["binary_scores"].numpy()
        binary_boxes = binary_result["boxes"].numpy()
        binary_boxes = np.concatenate([binary_boxes[np.arange(pnum)], binary_boxes[np.arange(pnum, pnum * 2)]], axis=1)
        ious = []
        for jj in range(64):
            ious.append(calc_hit(b, binary_boxes[jj]))
        ious = np.array(ious)
        matched_idx = np.argsort(-ious)[0]
        binary.append(binary_scores[matched_idx])
    binary = np.array(binary)
    a["binary"][i] = binary

for key in a.keys():
    for i in range(80):
        a[key][i] = np.array(a[key][i])
pickle.dump(a, open('./exp/%s/matched_result.pkl'%exp_dir_binary, 'wb'))

In [55]:
### nis
a = pickle.load(open("./exp/%s/matched_result.pkl"%exp_dir_binary, "rb"))
nms_index = pickle.load(open("./exp/%s/nms_index.pkl"%exp_dir, "rb"))
nis_fac = pickle.load(open("./exp/%s/nis_thres.pkl"%exp_dir_binary, "rb"))

ap  = np.zeros(600)
rec = np.zeros(600)
ap_ko  = np.zeros(600)
rec_ko = np.zeros(600)

for i in tqdm.trange(80):
    if len(a['scores'][i]) == 0:
        continue
    begin = obj_range[i][0] - 1
    end   = obj_range[i][1]

    score = a['scores'][i] * a['os'][i][:, None]
    bbox  = a['bboxes'][i]
    key   = a['keys'][i]
    binary = a['binary'][i]

    score, bbox, key = score[nms_index[i]], bbox[nms_index[i]], key[nms_index[i]]

    ko_mask = []
    for hoi_id in range(begin, end):
        gt_bbox = pickle.load(open('./data/hico_20160224_det/annotations/gt_hoi.pkl', 'rb'))[hoi_id]
        ko_mask += list(gt_bbox.keys())
    ko_mask = set(ko_mask)

    for hoi_id in range(begin, end):
        binary_mask = np.where(binary > nis_fac[hoi_id])[0]
        cur_score, cur_bbox, cur_key = score[binary_mask], bbox[binary_mask], key[binary_mask]
        output = calc_ap_ko(cur_score, cur_bbox, cur_key, hoi_id, begin, ko_mask)
        ap[hoi_id], rec[hoi_id] = output["ap"], output["rec"]
        ap_ko[hoi_id], rec_ko[hoi_id] = output["ap_ko"], output["rec_ko"]

100%|██████████| 80/80 [01:21<00:00,  1.02s/it]


In [57]:
### final HOI mAP
print("Default -- Full: %.2f, Rare: %.2f, Non-Rare: %.2f"%(100 * np.mean(ap), 100 * np.mean(ap[rare_index-1]), 100 * np.mean(np.array([v for i,v in enumerate(ap) if i+1 not in rare_index]))))
print("Known Object -- Full: %.2f, Rare: %.2f, Non-Rare: %.2f"%(100 * np.mean(ap_ko), 100 * np.mean(ap_ko[rare_index-1]), 100 * np.mean(np.array([v for i,v in enumerate(ap_ko) if i+1 not in rare_index]))))

Default -- Full: 35.15, Rare: 33.71, Non-Rare: 35.58
Known Object -- Full: 37.56, Rare: 35.87, Non-Rare: 38.06
